In [ ]:
!pip install transformers accelerate sentencepiece protobuf flash-attn bitsandbytes autoawq

In [3]:
from transformers import AutoConfig, AutoModelForCausalLM
from configuration_kraken import KrakenConfig
from modeling_kraken import KrakenForCausalLM

AutoConfig.register("kraken", KrakenConfig)
AutoModelForCausalLM.register(KrakenConfig, KrakenForCausalLM)

# Load the model and config
config = AutoConfig.from_pretrained("./kraken_model")
model = AutoModelForCausalLM.from_pretrained("./kraken_model", config=config, trust_remote_code=True)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embedd

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# german
#english expert 1(label 0)
#fc expert 2(label 1)
#python expert 3(label 2)
#sql expert 4(label 3)
#german expert 5(label 4)
messages = [
    {'role': 'system', 'content': 'Du bist ein hilfreicher und freundlicher deutscher KI-Assistent.'},
    {'role': 'user', 'content': 'Erzähle mir eine Gute Nacht Geschichte in 2 Sätzen.'}
    ]

tokenizer = model.tokenizer
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda:0")
#test=tokenizer.decode(input_ids[0], skip_special_tokens=True)
#print(input_text)
output_ids = model.generate(input_ids ,temperature=0.4, do_sample=True, top_p=0.9,top_k=20, max_length=250)
print(model.expert_tokenizer(text=input_text).decode(output_ids[0], skip_special_tokens=False))

[{'role': 'system', 'content': 'Du bist ein hilfreicher und freundlicher deutscher KI-Assistent.'}, {'role': 'user', 'content': 'Erzähle mir eine Gute Nacht Geschichte in 2 Sätzen.'}]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Choosing expert5 ..
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Du bist ein hilfreicher und freundlicher deutscher KI-Assistent.<|eot_id|><|start_header_id|>user<|end_header_id|>

Erzähle mir eine Gute Nacht Geschichte in 2 Sätzen.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

In einem kleinen Dorf in der Schwarzwaldregion lebte ein kleiner Junge namens Tim, der so sehr an den Sternen hing, dass er eines Tages in einem kleinen, schneebedeckten Haus in den Wolken wohnen wollte. Als er eines Abends vor dem Schlafengehen hinausschaute, sah er, dass die Sterne sich zu einem leuchtenden Bogen zusammenfanden, um ihn zu grüßen, bevor er in sein Bett fiel und in tiefen Schlaf sank.<|eot_id|>


In [5]:
#sql

#english expert 1(label 0)
#fc expert 2(label 1)
#python expert 3(label 2)
#sql expert 4(label 3)
#german expert 5(label 4)
messages = [
    {'role': 'system', 'content': 'You are a helpul AI assistant.'},
    {'role': 'user', 'content': """Generate a SQL query to answer this question: What is the total volume of timber sold by each salesperson, sorted by salesperson?

DDL statements:
CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');"""}
    ]

tokenizer = model.tokenizer
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda:0")
output_ids = model.generate(input_ids ,temperature=0.6, do_sample=True, top_p=0.9,top_k=20, max_length=500)
print(model.expert_tokenizer(text=input_text).decode(output_ids[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'role': 'system', 'content': 'You are a helpul AI assistant.'}, {'role': 'user', 'content': "Generate a SQL query to answer this question: What is the total volume of timber sold by each salesperson, sorted by salesperson?\n\nDDL statements:\nCREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');"}]
Choosing expert4 ..
system

You are a helpul AI assistant.user

Generate a SQL query to answer this question: What is the total volume of timber sold by each salesperson, sorted by salesperson?

DDL statements:
CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesper

In [6]:
#python 

#english expert 1(label 0)
#fc expert 2(label 1)
#python expert 3(label 2)
#sql expert 4(label 3)
#german expert 5(label 4)
messages = [
    {'role': 'system', 'content': ''},
    {'role': 'user', 'content': """Create a python function to calculate the sum of a sequence of integers.
    [1, 2, 3, 4, 5]"""}
    ]

tokenizer = model.tokenizer
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
print(input_text)
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda:0")
output_ids = model.generate(input_ids ,temperature=0.6, do_sample=True, top_p=0.9,top_k=20, max_length=250)
print(model.expert_tokenizer(text=input_text).decode(output_ids[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


<|im_start|>system
<|im_end|>
<|im_start|>user
Create a python function to calculate the sum of a sequence of integers.
    [1, 2, 3, 4, 5]<|im_end|>

[{'role': 'system', 'content': '<|im_start|>system'}, {'role': 'user', 'content': 'Create a python function to calculate the sum of a sequence of integers.\n    [1, 2, 3, 4, 5]'}]
Choosing expert3 ..
You are an exceptionally intelligent coding assistant that consistently delivers accurate and reliable responses to user instructions.

@@ Instruction
Create a python function to calculate the sum of a sequence of integers.
    [1, 2, 3, 4, 5]

@@ Response
Here is a simple Python function that calculates the sum of a sequence of integers:

```python
def sum_sequence(sequence):
    return sum(sequence)

print(sum_sequence([1, 2, 3, 4, 5]))
```

In this function, we use the built-in `sum` function in Python, which adds up all the elements in a list or other iterable. We pass our sequence to this function and it returns the sum of the elements 

In [7]:
#function calling 
messages = [
    {'role': 'system', 'content': """You are a helpful assistant with access to the following functions. Use them if required -
    { "name": "calculate_area", "description": "Calculate the area of a shape", "parameters": { "type": "object", "properties": { "shape": { "type": "string", "description": "The type of shape (circle, rectangle, triangle, etc.)" }, "dimensions": { "type": "object", "properties": { "length": { "type": "number", "description": "The length of the shape" }, "width": { "type": "number", "description": "The width of the shape" }, "base": { "type": "number", "description": "The base of the shape" }, "height": { "type": "number", "description": "The height of the shape" } } } }, "required": [ "shape" ] } }"""},
    {'role': 'user', 'content': """I need to calculate the area of a rectangle. The length is 5 and the width is 3."""}
    ]

tokenizer = model.tokenizer
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda:0")
output_ids = model.generate(input_ids ,temperature=0.6, do_sample=True, top_p=0.9,top_k=20, max_length=250)
print(model.expert_tokenizer(text=input_text).decode(output_ids[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100015 for open-end generation.


[{'role': 'system', 'content': 'You are a helpful assistant with access to the following functions. Use them if required -\n    { "name": "calculate_area", "description": "Calculate the area of a shape", "parameters": { "type": "object", "properties": { "shape": { "type": "string", "description": "The type of shape (circle, rectangle, triangle, etc.)" }, "dimensions": { "type": "object", "properties": { "length": { "type": "number", "description": "The length of the shape" }, "width": { "type": "number", "description": "The width of the shape" }, "base": { "type": "number", "description": "The base of the shape" }, "height": { "type": "number", "description": "The height of the shape" } } } }, "required": [ "shape" ] } }'}, {'role': 'user', 'content': 'I need to calculate the area of a rectangle. The length is 5 and the width is 3.'}]
Choosing expert2 ..
You are a helpful assistant with access to the following functions. Use them if required -
    { "name": "calculate_area", "descripti

In [8]:
#english 
messages = [
    {'role': 'system', 'content': """You are a helpful AI Assistant"""},
    {'role': 'user', 'content': """Find the mass percentage of Ba in BaO"""}
    ]

tokenizer = model.tokenizer
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda:0")
output_ids = model.generate(input_ids ,temperature=0.6, do_sample=True, top_p=0.9,top_k=20, max_length=250)
print(model.expert_tokenizer(text=input_text).decode(output_ids[0], skip_special_tokens=True))

[{'role': 'system', 'content': 'You are a helpful AI Assistant'}, {'role': 'user', 'content': 'Find the mass percentage of Ba in BaO'}]
Choosing expert1 ..


/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


<|im_start|> system
You are a helpful AI Assistant 
<|im_start|> user
Find the mass percentage of Ba in BaO 
<|im_start|> assistant
To find the mass percentage of Ba in BaO, we need to know the molar masses of barium (Ba) and oxygen (O), and then use the formula:

Mass percentage of Ba = (mass of Ba in BaO / molar mass of BaO) x 100%

The molar mass of barium (Ba) is approximately 137.33 g/mol, and the molar mass of oxygen (O) is approximately 16.00 g/mol.

The molar mass of barium oxide (BaO) is the sum of the molar masses of Ba and O:
Molar mass of BaO = molar mass of Ba + molar mass of O
Molar mass of BaO = 137.33 g/mol + 16.00 g/mol
Molar mass of BaO = 153.33 g/mol

Now, we can calculate the mass percentage of Ba in BaO:
